In [1]:
%%time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from Preprocess import events_to_df, scale_shift_feats, for_nn, events_to_df_test
from Models import cnn, lstm, lstm_big, train
from Compare import compareAB

Wall time: 5.21 s


In [2]:
max_ev = int(1e5)
n_constits = 20
sort = "PT"
j1_path = "Results1.txt"
j2_path = "Results2.txt"
j1 = events_to_df_test(j1_path, label=0, max_ev=max_ev, n_constits=n_constits, sort=sort)
j2 = events_to_df_test(j2_path, label=0, max_ev=max_ev, n_constits=n_constits, sort=sort).rename(columns={"track_PT": "track_posx", "track_Eta": "track_posy", "track_Phi": "track_posz", "track_DeltaR": "track_momx", "track_PID": "track_momy", "track_D0": "track_momz", "track_DZ": "track_vertx", "track_errD0": "track_verty", "track_errDZ": "track_vertz"  })

In [4]:
j1["mult"] = j1["track_PT"].map(lambda x: x.nonzero()[0].shape[0])
j2["mult"] = j2["track_posx"].map(lambda x: x.nonzero()[0].shape[0])

In [10]:
j1.loc[0:6].head(10)

,Event,Jet,MET,jet_PT,jet_Eta,jet_Phi,track_PT,track_Eta,track_Phi,track_DeltaR,track_PID,track_D0,track_DZ,track_errD0,track_errDZ,Charge,label,mult
0,1,2,13.0059,154.345,0.399627,1.355850,"[20.9009, 11.0109, 10.8936, 9.16738, 6.94298, ...","[0.326518, 0.543478, 0.463958, 0.483792, 0.181...","[1.43039, 1.11864, 1.33275, 1.21662, 1.54282, ...","[0.104413, 0.277418, 0.0683499, 0.162684, 0.28...","[211.0, -2212.0, -321.0, 2212.0, -211.0, 211.0...","[-0.0123897, 0.0187548, 0.0131966, 0.0111784, ...","[-0.309593, -0.00715879, -0.0106778, 0.0170454...","[0.0189313, 0.0208891, 0.019892, 0.0263278, 0....","[0.035, 0.029978, 0.0299762, 0.0363115, 0.0670...","[1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1....",0,17
1,2,1,40.6203,156.246,0.711259,2.395520,"[31.7769, 24.7433, 13.241, 6.01747, 2.55923, 0...","[0.716798, 0.727803, 0.716285, 0.723045, 0.689...","[2.3971, 2.38298, 2.39691, 2.36813, 2.25307, 0...","[0.00576117, 0.0207545, 0.00521603, 0.0298175,...","[321.0, -211.0, 211.0, -321.0, 211.0, 0.0, 0.0...","[-0.0386863, -0.0160853, 0.0614451, -0.069298,...","[0.049211, -0.0462333, 0.114672, 0.0138826, -0...","[0.0200087, 0.0210738, 0.0225769, 0.057738, 0....","[0.035, 0.035, 0.035, 0.0776417, 0.104879, 0.0...","[1.0, -1.0, 1.0, -1.0, 1.0, 0.0, 0.0, 0.0, 0.0...",0,5
2,4,1,10.9612,146.883,-0.618444,-2.485670,"[15.6784, 11.824, 9.75391, 7.84842, 7.70134, 7...","[-0.613276, -0.607632, -0.598817, -0.591783, -...","[-2.4687, -2.47467, -2.4805, -2.39173, -2.4380...","[0.0177443, 0.015424, 0.0202967, 0.0976507, 0....","[-211.0, 211.0, -321.0, 211.0, -211.0, 211.0, ...","[-0.0292481, -0.0542348, -0.0261624, 1.05176, ...","[-0.0344169, -0.148685, -0.00210626, -0.412155...","[0.0212926, 0.0217777, 0.0224228, 0.0375934, 0...","[0.0345485, 0.0348582, 0.0316432, 0.0492338, 0...","[-1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1....",0,14
3,4,2,10.9612,141.701,1.336580,0.911058,"[13.8469, 10.4313, 7.92871, 5.57126, 5.11349, ...","[1.19325, 1.35865, 1.41081, 0.82712, 1.20425, ...","[0.890453, 0.905353, 0.890546, 0.880854, 0.721...","[0.144807, 0.0227954, 0.0770055, 0.510359, 0.2...","[-321.0, 211.0, -321.0, -211.0, -211.0, 211.0,...","[0.0306925, 0.0859371, -0.209385, -0.038947, -...","[-0.0724166, 0.817543, -0.0650884, -0.100067, ...","[0.0234174, 0.0269202, 0.0512195, 0.0605993, 0...","[0.0493264, 0.0549251, 0.104342, 0.0888841, 0....","[-1.0, 1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -...",0,13
4,8,2,107.7310,147.981,1.090440,0.872050,"[67.3276, 7.93785, 7.4719, 6.56852, 5.62262, 4...","[0.983134, 1.32853, 0.984039, 1.47055, 0.94015...","[0.789361, 1.19115, 0.847358, 1.16485, 0.91372...","[0.135473, 0.398133, 0.109232, 0.479805, 0.155...","[-321.0, 211.0, 211.0, 211.0, 11.0, 211.0, 211...","[-0.0676547, 9.29379, 0.453631, -0.0024535, 0....","[0.0450158, -7.19366, 0.0451834, 0.0814825, 0....","[0.015488, 0.0505654, 0.0452786, 0.0710634, 0....","[0.0366999, 0.0964474, 0.0709379, 0.136899, 0....","[-1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1....",0,18
5,10,2,36.4178,141.493,-1.724620,1.413570,"[18.2058, 13.9348, 4.91668, 4.33706, 3.96889, ...","[-1.60832, -1.68705, -1.56892, -1.76455, -1.52...","[1.47926, 1.41275, 1.34104, 1.41166, 1.40206, ...","[0.133569, 0.037577, 0.171753, 0.0399807, 0.20...","[2212.0, 211.0, -211.0, -211.0, -211.0, 211.0,...","[0.0438751, 0.358078, 0.142217, 0.161676, -9.8...","[-0.0285255, -2.59256, 0.221538, 1.88059, -5.6...","[0.0255038, 0.0262996, 0.0968647, 0.116769, 0....","[0.0685946, 0.0693421, 0.20086, 0.282744, 0.22...","[1.0, 1.0, -1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -...",0,13
6,13,1,70.0995,148.972,0.307656,0.657164,"[25.191, 16.0247, 13.6971, 10.6429, 4.32376, 3...","[0.322425, 0.309264, 0.248053, 0.286772, 0.296...","[0.602996, 0.687066, 0.636094, 0.636062, 0.633...","[0.0561456, 0.0299451, 0.0632174, 0.0296887, 0...","[-211.0, -211.0, -321.0, 211.0, 211.0, -211.0,...","[-0.244949, -0.00406774, 0.102736, 0.142773, 0...","[-0.345783, -0.0042972, 0.37, 0.069454, 0.0738...","[0.0184754, 0.019382, 0.0196338, 0.0199192, 0...."

In [11]:
j2.loc[0:6].head(10)

,Event,Jet,MET,jet_PT,jet_Eta,jet_Phi,track_posx,track_posy,track_posz,track_momx,track_momy,track_momz,track_vertx,track_verty,track_vertz,Charge,label,mult
0,1,2,13.0059,154.345,0.399627,1.355850,"[0.0313671, 0.0110765, 0.00837746, 0.00522823,...","[-0.000877778, -0.00397269, 0.000367612, -0.00...","[0.0154881, -0.00850886, 0.0221487, 0.00533325...","[-0.194216, -0.289199, 0.0422858, -0.326712, -...","[-6.94026, -0.806338, -0.963645, -1.1468, -9.9...","[-1.26532, -0.00544288, -0.365424, -0.430231, ...","[0.0239016, 0.0239016, 0.0239016, 0.0239016, 0...","[0.043338, 0.043338, 0.043338, 0.043338, 0.043...","[-0.00930033, -0.00930033, -0.00930033, -0.009...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0...",0,17
1,2,1,40.6203,156.246,0.711259,2.395520,"[0.00439476, 0.00416435, -0.00110653, -0.00262...","[0.00357045, 0.00451809, -0.00116746, -0.00284...","[-0.0007764, 0.0114672, -0.00462333, 0.0049211...","[-1.61375, -9.73619, 17.9586, -23.3698, 4.3054...","[1.98632, 8.9739, -17.0212, 21.532, -4.20392, ...","[1.90622, 10.3164, -19.6407, 24.7789, -4.74005...","[-0.0752157, -0.0752157, -0.0752157, -0.075215...","[0.0662454, 0.0662454, 0.0662454, 0.0662454, 0...","[0.114507, 0.114507, 0.114507, 0.114507, 0.114...","[1.0, 1.0, -1.0, 1.0, -1.0, 0.0, 0.0, 0.0, 0.0...",0,5
2,4,1,10.9612,146.883,-0.618444,-2.485670,"[0.0063709, 0.00624448, 0.00438852, 0.0033548,...","[-0.00916662, -0.00695227, -0.00742557, -0.004...","[0.00831079, -0.000994521, 0.0188629, -0.01486...","[-1.18772, 1.33828, -2.97004, -9.29044, 12.260...","[-0.825482, 1.20204, -1.7553, -7.31396, 9.7716...","[-0.901934, 1.34639, -2.28347, -7.63497, 10.22...","[-2.07951, -2.07951, -2.07951, -2.07951, -2.07...","[-1.57782, -1.57782, -1.57782, -1.57782, -1.57...","[-1.75618, -1.75618, -1.75618, -1.75618, -1.75...","[1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1....",0,14
3,4,2,10.9612,141.701,1.336580,0.911058,"[8.29439, 0.643003, 0.00676019, 0.00562746, 0....","[-5.04495, -0.53157, -0.00530583, -0.00717553,...","[8.78637, 5.76792, 0.0817543, 0.0174931, -0.01...","[0.395498, 0.575766, 6.44034, -0.300171, -0.48...","[0.650239, 0.696465, 8.20568, -0.235412, -0.46...","[0.988499, 0.714869, 18.9533, -0.587689, -0.69...","[11.874, 11.874, 11.874, 11.874, 11.874, 11.87...","[12.5331, 12.5331, 12.5331, 12.5331, 12.5331, ...","[27.013, 27.013, 27.013, 27.013, 27.013, 27.01...","[1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, ...",0,13
4,10,2,36.4178,141.493,-1.724620,1.413570,"[1.52651, 0.0353615, 0.0190668, 0.0159633, 0.0...","[0.0112552, -0.00563583, -0.00471824, -0.00256...","[0.45384, -0.259256, -0.0155693, 0.188059, 0.0...","[-0.0192571, 2.19322, -0.301917, -0.687289, -1...","[2.61179, 13.7612, -1.22007, -4.28225, -4.7874...","[-5.87522, -36.3591, 2.9461, 12.2905, 11.2917,...","[0.782776, 0.782776, 0.782776, 0.782776, 0.782...","[5.13756, 5.13756, 5.13756, 5.13756, 5.13756, ...","[-14.0246, -14.0246, -14.0246, -14.0246, -14.0...","[1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1...",0,13
5,13,1,70.0995,148.972,0.307656,0.657164,"[0.00848116, 0.00610308, 0.00474929, 0.0013329...","[-0.0114852, -0.00826428, -0.00645577, -0.0018...","[0.0069454, 0.037, -0.0112269, 0.00738339, -0....","[8.56158, -11.0182, -2.79277, 3.48537, -12.388...","[6.32222, -8.13684, -2.05454, 2.55872, -10.164...","[3.09409, -3.43255, -1.03835, 1.29871, -5.0352...","[0.5287, 0.5287, 0.5287, 0.5287, 0.5287, 0.528...","[0.404352, 0.404352, 0.404352, 0.404352, 0.404...","[0.201506, 0.201506, 0.201506, 0.201506, 0.201...","[1.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -...",0,12
6,15,1,28.3617,151.055,-1.503580,-1.581340,"[0.0239258, 0.0171836, 0.0168915, 0.0118412, 0...","[-9.82392e-05, -0.000402526, -0.00518887, 0.00...","[-0.0379752, 0.0171635, 0.00957379, 0.0356301,...","[-0.0288295, 0.0619769, -0.356628, 0.0811322, ...","[-7.02132, 2.64576, -1.16094, -3.22443, 0.6443...","[-13.4204, 5.1998, -2.70669, -5.94096, 1.12806...","[0.0388261, 0.0388261, 0.0388261, 0.0388261, 0...","[0.0738702, 0.0738702, 0.0738702, 0.0738

In [5]:
j1.loc[13:16].head()

,Event,Jet,MET,jet_PT,jet_Eta,jet_Phi,track_PT,track_Eta,track_Phi,track_DeltaR,track_PID,track_D0,track_DZ,track_errD0,track_errDZ,Charge,label
13,21,2,23.8545,152.066,-1.113620,0.125269,"[21.3945, 8.5016, 8.23295, 6.80897, 5.27595, 5...","[-1.02409, -1.0247, -0.944901, -1.00494, -1.26...","[0.145191, 0.120457, 0.122403, 0.0891206, 0.16...","[0.0917255, 0.0890581, 0.168748, 0.114534, 0.1...","[-321.0, 321.0, 211.0, 211.0, -211.0, 321.0, -...","[0.078586, -0.0128973, 4.48901, 0.0129712, 0.0...","[0.044937, -0.105403, 0.0276958, 0.126668, -0....","[0.0223199, 0.0386521, 0.0390535, 0.0546965, 0...","[0.0437088, 0.0726444, 0.0619437, 0.0922885, 0...","[-1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1...",0
14,22,1,23.5648,147.404,-2.213930,0.170596,"[13.4755, 4.72749, 3.93303, 3.41016, 3.10152, ...","[-2.22928, -2.19997, -1.63579, -2.17424, -2.05...","[0.201679, 0.137638, 0.235418, 0.179964, 0.265...","[0.0346655, 0.035794, 0.581761, 0.0407827, 0.1...","[321.0, 211.0, -211.0, -211.0, -211.0, -211.0,...","[-0.00980991, 0.31394, 0.173876, 1.16978, 0.14...","[0.0751543, -1.71022, -0.732226, -5.0053, 0.64...","[0.0344334, 0.134614, 0.116146, 0.149794, 0.14...","[0.147796, 0.602799, 0.259194, 0.589689, 0.484...","[1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, ...",0
15,24,1,16.7453,151.713,0.735385,1.264480,"[25.7625, 23.31, 14.6248, 9.70673, 5.22164, 3....","[0.698175, 0.708067, 0.756698, 0.73083, 0.7365...","[1.2267, 1.25152, 1.43821, 1.19614, 1.32, 1.15...","[0.0530335, 0.0302364, 0.175028, 0.0684978, 0....","[211.0, -211.0, -2212.0, -211.0, -211.0, -11.0...","[0.0376519, 0.0393497, -0.0151474, -0.0585947,...","[0.0804993, -0.0299654, 0.0205764, -0.00069432...","[0.0200854, 0.0211586, 0.0223909, 0.0240324, 0...","[0.0337781, 0.035, 0.035, 0.035884, 0.0859656,...","[1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, ...",0
16,24,2,16.7453,141.684,0.366800,-2.018880,"[62.0578, 23.1302, 23.0614, 21.2901, 16.0209, ...","[0.373972, 0.327162, 0.414832, 0.35049, 0.3747...","[-1.96232, -1.94867, -1.97373, -1.96984, -1.98...","[0.0570091, 0.0806242, 0.0659207, 0.0516745, 0...","[211.0, 321.0, 211.0, 13.0, -211.0, 321.0, -32...","[0.052102, 0.267333, -0.0412006, -0.0119062, -...","[-0.0843732, 0.436853, -0.537552, 0.0796878, -...","[0.0148183, 0.0186595, 0.0187194, 0.0187125, 0...","[0.035, 0.035, 0.0297183, 0.035, 0.035, 0.0872...","[1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 0...",0


In [6]:
j2.loc[13:16].head()

,Event,Jet,MET,jet_PT,jet_Eta,jet_Phi,track_posx,track_posy,track_posz,track_momx,track_momy,track_momz,track_vertx,track_verty,track_vertz,Charge,label
13,22,1,23.56480,147.404,-2.21393,0.170596,"[0.0898249, 0.0209385, 0.00672637, 0.00430736,...","[-0.2136, -0.115089, -0.0280969, -0.0310971, -...","[-0.650749, -0.50053, 0.013437, -0.171022, -0....","[-2.73793, -3.35508, -0.716703, 4.68278, -3.82...","[-1.15138, -0.6104, -0.171578, 0.648628, -0.91...","[13.5504, 14.8032, 2.51425, -21.0702, 9.71209,...","[0.443681, 0.443681, 0.443681, 0.443681, 0.443...","[0.0283043, 0.0283043, 0.0283043, 0.0283043, 0...","[-2.11243, -2.11243, -2.11243, -2.11243, -2.11...","[-1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, ...",0
14,24,2,16.74530,141.684,0.36680,-2.018880,"[0.0113492, 0.00379011, 0.00109707, 0.00082400...","[-0.00498742, -0.00161554, -0.000462602, -0.00...","[-0.00767347, -0.0537552, 0.00796878, 0.003476...","[6.44547, -9.04273, 8.27204, -0.561379, -0.070...","[14.6671, -21.2146, 19.6174, -2.61152, 0.43319...","[-6.14536, 9.84336, -7.61567, 1.30597, -0.2640...","[-0.52623, -0.52623, -0.52623, -0.52623, -0.52...","[-1.18648, -1.18648, -1.18648, -1.18648, -1.18...","[0.430088, 0.430088, 0.430088, 0.430088, 0.430...","[-1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 0...",0
15,29,1,10.57780,141.958,1.57988,1.824500,"[0.0216841, 0.00319125, 0.00304364, 0.00019114...","[0.00693088, 0.000995688, 0.000540058, 4.29212...","[-0.0245719, -0.0274927, -0.00329103, -0.00994...","[0.655029, 0.942521, 1.25552, -4.81033, 1.2818...","[-2.04934, -3.02084, -7.0758, 21.4228, -6.1951...","[-4.7044, -7.13457, -14.3504, 52.3881, -14.455...","[-0.137645, -0.137645, -0.137645, -0.137645, -...","[-0.0583033, -0.0583033, -0.0583033, -0.058303...","[0.153213, 0.153213, 0.153213, 0.153213, 0.153...","[-1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0,...",0
16,30,2,3.23641,143.625,-1.17613,1.878430,"[7.39535, 0.0833774, 0.0120994, 0.0116257, 0.0...","[3.286, 0.0279771, 0.00556646, 0.00255014, 0.0...","[-3.28139, 0.0252968, 0.0131997, -0.00201338, ...","[-1.36765, 0.655627, -0.334916, -0.231534, 0.6...","[3.07798, -1.9539, 0.727984, 1.05553, -0.94279...","[-5.02335, 3.08789, -0.964617, -1.77031, 0.981...","[-10.6293, -10.6293, -10.6293, -10.6293, -10.6...","[37.5857, 37.5857, 37.5857, 37.5857, 37.5857, ...","[-54.5586, -54.5586, -54.5586, -54.5586, -54.5...","[1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1....",0


In [24]:
j2["track_PT"] = ((np.power(j2.track_momx,2)+np.power(j2.track_momy,2))*np.power(j2.Charge,2)).map(np.sqrt)
j2["track_DZ"] = j2.track_posz*10
j2["track_D0"] = ((np.power(j2.track_posx,2)+np.power(j2.track_posy,2))*100).map(np.sqrt)

j2["track_Theta"] = j2[["track_PT", "track_momz", "Charge"]].apply(lambda x: np.arctan2(x.track_PT,x.track_momz*x.Charge), axis=1)
j2["track_Eta"] = -((0.5*j2.track_Theta).map(np.tan)).map(np.log)
def InfToZero(x):
    x = x.copy()
    x[(x==np.inf) | (x==-np.inf)] = 0
    return x
j2["track_Eta"] = j2.track_Eta.map(InfToZero)

j2["track_Phi"] = j2[["track_momx", "track_momy", "Charge"]].apply(lambda x: np.arctan2(x.track_momy*x.Charge,x.track_momx*x.Charge), axis=1)